## Run

### Split system in EMT, using decoupling node

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

# Define lists for timestep and prefix
delays=(0.0001, 0.0002, 0.0004, 0.001, 0.003, 0.01)
prefixes=(linear_1e-4 linear_2e-4 linear_4e-4 linear_0.001 linear_0.003 linear_0.01)
i_intfs=(30.1080 30.216 30.432 31.08 33.24 40.8)

# Iterate over the lists
for i in "${!delays[@]}"; do
  EMT_RC_split_decoupled_node --timestep=0.00005 -o delay="${delays[$i]}" -o method=extrapolation-linear -o i-intf-0="${i_intfs[$i]}" -o prefix="${prefixes[$i]}"
done

## Validate Results

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib widget

H_v = np.array([1e-4, 2e-4, 4e-4, 0.001, 0.003, 0.01])
H_v_legends = ['1e-4', '2e-4', '4e-4', '0.001', '0.003', '0.01']
y_1_prev_H = [30.1080, 30.216, 30.432, 31.08, 33.24, 40.8]

results_emt = rt.read_timeseries_dpsim('logs/EMT_RC_monolithic/EMT_RC_monolithic.csv')

# results_emt_attributes_zoh_S1 = {}
# for H_v_leg in H_v_legends:
#     results_emt_attributes_zoh_S1[H_v_leg] = rt.read_timeseries_dpsim('logs/EMTCosimAttributes_zoh_' + H_v_leg + '_S1/S1.csv')

# results_emt_attributes_zoh_S2 = {}
# for H_v_leg in H_v_legends:
#     results_emt_attributes_zoh_S2[H_v_leg] = rt.read_timeseries_dpsim('logs/EMTCosimAttributes_zoh_' + H_v_leg + '_S2/S2.csv')

results_emt_itm_linear = {}
for H_v_leg in H_v_legends:
    results_emt_itm_linear[H_v_leg] = rt.read_timeseries_dpsim('logs/EMT_RC_split_decoupled_linear_' + H_v_leg + '_0_0/EMT_RC_split_decoupled_linear_' + H_v_leg + '_0_0.csv')

pt.set_timeseries_labels(results_emt['v_2'], 'v_2 - Monolithic')
pt.plot_timeseries('Co-simulation results - v_intf - linear', results_emt['v_2'])
plt.xlabel('t [s]')
plt.ylabel('Interface voltage')
plt.grid()

pt.set_timeseries_labels(results_emt['i_rline'], 'i_rline - Monolithic')
pt.plot_timeseries('Co-simulation results - i_intf - linear', results_emt['i_rline'])

for k, results_att_list in results_emt_itm_linear.items():
    pt.set_timeseries_labels(results_att_list['v_intf'], 'v_intf - S_1, linear')
    pt.plot_timeseries('Co-simulation results - v_intf - linear', results_att_list['v_intf'], '--')
    
    pt.set_timeseries_labels(results_att_list['i_intf'], 'i_intf - S_1, linear')
    pt.plot_timeseries('Co-simulation results - i_intf - linear', results_att_list['i_intf'], '--')

plt.xlabel('t [s]')
plt.ylabel('Interface current')
plt.grid()
plt.show()

In [3]:
# path_split_emt = 'logs/EMT_RC_split_decoupled_delay_0_0_0/'
# logName_split_emt = 'EMT_RC_split_decoupled_delay_0_0_0'
# logFilename_split_emt = path_split_emt + logName_split_emt + '.csv'
# print(logFilename_split_emt)

# path_mono_emt = 'logs/EMT_RC_monolithic/'
# logName_mono_emt = 'EMT_RC_monolithic'
# logFilename_mono_emt = path_mono_emt + logName_mono_emt + '.csv'
# print(logFilename_mono_emt)

# decoupled_emt_split = rt.read_timeseries_dpsim(logFilename_split_emt)
# mono_emt = rt.read_timeseries_dpsim(logFilename_mono_emt)

# pt.set_timeseries_labels(mono_emt['v_2'], 'v_2 Monolithic')
# pt.plot_timeseries('Co-simulation results - v_intf', mono_emt['v_2'])
# pt.set_timeseries_labels(decoupled_emt_split['v_2_1'], 'v_2 decoupled S1')
# pt.plot_timeseries('Co-simulation results - v_intf', decoupled_emt_split['v_2_1'], '--')
# pt.set_timeseries_labels(decoupled_emt_split['v_2_2'], 'v_2 decoupled S2')
# pt.plot_timeseries('Co-simulation results - v_intf', decoupled_emt_split['v_2_2'], '-.')
# # plt.title('WSCC-09 with 3 lines replaced by decoupling equivalents')
# plt.xlabel('Time [s]')
# plt.ylabel('Voltage [V]')

# plt.show()

In [4]:
# pt.set_timeseries_labels(mono_emt['i_rline'], 'i_rline Monolithic')
# pt.plot_timeseries('Co-simulation results - i_intf', mono_emt['i_rline'])
# pt.set_timeseries_labels(decoupled_emt_split['i_ref'], 'i_intf, decoupled')
# pt.plot_timeseries('Co-simulation results - i_intf', decoupled_emt_split['i_ref'], '--')
# plt.xlabel('Time [s]')
# plt.ylabel('Voltage [V]')

# plt.show()

In [ ]:
time_step = 50e-6

global_error_zoh = []
global_error_linear = []
global_error_delay = []

ts_v1_m = results_emt['v_1']
ts_v2_m = results_emt['v_2']
ts_i_intf_m = results_emt['i_rline']

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 6))

for i in range(0, len(H_v_legends)):    
    H = H_v[i]
    
    m = int(round(H/time_step))

    # ts_v1_zoh = results_emt_attributes_zoh_S1[H_v_legends[i]]['v_1']
    # ts_v2_zoh = results_emt_attributes_zoh_S1[H_v_legends[i]]['v_2']
    # ts_i_intf_zoh = results_emt_attributes_zoh_S1[H_v_legends[i]]['i_intf']
    
    ts_v1_linear = results_emt_itm_linear[H_v_legends[i]]['v_1']
    ts_v2_linear = results_emt_itm_linear[H_v_legends[i]]['v_2_1']
    ts_i_intf_linear = results_emt_itm_linear[H_v_legends[i]]['i_intf']

    # if num_vs == 0:
    #     ts_1_m = results_emt[1]
    #     ts_2_m = results_emt[2]

    #     ts_1_zoh = results_emt_attributes_zoh[H_v_legends[i]][1]
    #     ts_2_zoh = results_emt_attributes_zoh[H_v_legends[i]][2]
        
    #     ts_1_linear = results_emt_attributes_linear_S1[H_v_legends[i]][1]
    #     ts_2_linear = results_emt_attributes_linear_S1[H_v_legends[i]][2]
    # else:
    #     ts_1_m = results_emt[2]
    #     ts_2_m = results_emt[3]
    
    #     ts_1_zoh = results_emt_attributes_zoh[H_v_legends[i]][2]
    #     ts_2_zoh = results_emt_attributes_zoh[H_v_legends[i]][3]
        
    #     ts_1_linear = results_emt_attributes_linear_S1[H_v_legends[i]][2]
    #     ts_2_linear = results_emt_attributes_linear_S1[H_v_legends[i]][3]
    
    xy_a = np.array([ts_v1_m.values[::m], ts_v2_m.values[::m], ts_i_intf_m.values[::m], ts_v2_m.values[::m]])
    # xy_zoh = np.array([ts_v1_zoh.values[::m], ts_v2_zoh.values[::m], ts_i_intf_zoh.values[::m], ts_v2_zoh.values[::m]])
    xy_linear = np.array([ts_v1_linear.values[::m], ts_v2_linear.values[::m], ts_i_intf_linear.values[::m], ts_v2_linear.values[::m]])
    
    # global_error_v1_zoh_t = np.abs(ts_v1_m.values[::m] - ts_v1_zoh.values[::m], axis=0)
    # global_error_v2_zoh_t = np.abs(ts_v2_m.values[::m] - ts_v2_zoh.values[::m], axis=0)
    # global_error_i_intf_zoh_t = np.abs(ts_i_intf_m.values[::m] - ts_i_intf_zoh.values[::m], axis=0)

    global_error_v1_linear_t = np.abs(ts_v1_m.values[::m] - ts_v1_linear.values[::m])
    global_error_v2_linear_t = np.abs(ts_v2_m.values[::m] - ts_v2_linear.values[::m])
    global_error_i_intf_linear_t = np.abs(ts_i_intf_m.values[::m] - ts_i_intf_linear.values[::m])
    
    # global_error_zoh_t = np.linalg.norm(xy_a - xy_zoh, axis=0)
    global_error_linear_t = np.linalg.norm(xy_a - xy_linear, axis=0)

    # plt.plot(ts_1_m.time[::m], gloal_error_zoh_t, label='ZOH - ' + H_v_legends[i])
    # plt.plot(ts_1_m.time[::m], gloal_error_linear_t, label='linear - ' + H_v_legends[i])

    axes[0].plot(ts_v1_m.time[::m], global_error_v1_linear_t, label='V1 - linear - ' + H_v_legends[i])
    axes[0].plot(ts_v2_m.time[::m], global_error_v2_linear_t, label='V2 - linear - ' + H_v_legends[i])

    axes[1].plot(ts_i_intf_m.time[::m], global_error_i_intf_linear_t, label='i_intf - linear - ' + H_v_legends[i])
    
    # global_error_zoh.append(np.max(global_error_zoh_t))
    global_error_linear.append(np.max(np.max(global_error_linear_t)))

axes[0].set_xlabel('t [s]')
axes[0].set_ylabel('Global error')
# plt.xlim((0, 0.4))
# plt.ylim((0.0008, 0.0012))
axes[0].grid()
axes[0].legend()

axes[1].set_xlabel('t [s]')
axes[1].set_ylabel('Global error')
# plt.xlim((0, 0.4))
# plt.ylim((0.0008, 0.0012))
axes[1].grid()
axes[1].legend()

In [ ]:
plt.figure()
# plt.plot(np.log10(H_v), np.log10(global_error_zoh), 'b', label='ZOH')
# plt.plot(np.log10(H_v), np.log10(global_error_zoh), 'bo')
plt.plot(np.log10(H_v), np.log10(np.multiply(H_v, 10*0.012)), 'k', label=r'$\mathcal{O}(H)$')
plt.plot(np.log10(H_v), np.log10(global_error_linear), 'r', label='Linear')
plt.plot(np.log10(H_v), np.log10(global_error_linear), 'ro')
plt.plot(np.log10(H_v), np.log10(np.multiply(H_v**2, 10**1.5)), '--k', label=r'$\mathcal{O}(H^2)$')
# plt.plot(np.log10(H_v), np.log10(global_error_delay), 'g', label='Delay')
# plt.plot(np.log10(H_v), np.log10(global_error_delay), 'go')
plt.xlabel('')
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()